In [48]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

prolongations = pd.read_csv('prolongations.csv')
financial_data = pd.read_csv('financial_data.csv')

prolongations.head()

,id,month,AM
0,42,ноябрь 2022,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна
4,429,ноябрь 2022,Соколова Анастасия Викторовна


In [49]:
financial_data.head()

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович


In [50]:
def month_to_number(month_str):
    month_dict = {
        'январь': 1, 'февраль': 2, 'март': 3, 'апрель': 4,
        'май': 5, 'июнь': 6, 'июль': 7, 'август': 8,
        'сентябрь': 9, 'октябрь': 10, 'ноябрь': 11, 'декабрь': 12
    }
    
    if isinstance(month_str, str):
        for month_name, month_num in month_dict.items():
            if month_name in month_str.lower():
                year_part = month_str.split()[-1] if len(month_str.split()) > 1 else '2023'
                return int(year_part), month_num
    return 2023, 1

In [51]:
prolongations_clean = prolongations.copy()
prolongations_clean[['year', 'month_num']] = prolongations_clean['month'].apply(
    lambda x: pd.Series(month_to_number(x))
)
prolongations_clean['end_date'] = pd.to_datetime(
    prolongations_clean['year'].astype(str) + '-' + 
    prolongations_clean['month_num'].astype(str) + '-01'
)

print("\nОбработанные prolongations:")
prolongations_clean[['id', 'month', 'year', 'month_num']].head()


Обработанные prolongations:


,id,month,year,month_num
0,42,ноябрь 2022,2022,11
1,453,ноябрь 2022,2022,11
2,548,ноябрь 2022,2022,11
3,87,ноябрь 2022,2022,11
4,429,ноябрь 2022,2022,11


In [52]:
month_columns = [col for col in financial_data.columns 
                if any(month in col.lower() for month in ['январь', 'февраль', 'март', 'апрель', 
                                                         'май', 'июнь', 'июль', 'август', 
                                                         'сентябрь', 'октябрь', 'ноябрь', 'декабрь'])]

month_columns

['Ноябрь 2022',
 'Декабрь 2022',
 'Январь 2023',
 'Февраль 2023',
 'Март 2023',
 'Апрель 2023',
 'Май 2023',
 'Июнь 2023',
 'Июль 2023',
 'Август 2023',
 'Сентябрь 2023',
 'Октябрь 2023',
 'Ноябрь 2023',
 'Декабрь 2023',
 'Январь 2024',
 'Февраль 2024']

In [53]:
import re


def clean_numeric_value_alt(val):
    if pd.isna(val):
        return 0
        
    if isinstance(val, (int, float)):
        return float(val)
        
    if isinstance(val, str):
        val_lower = val.lower().strip()
        
        if any(stop_word in val_lower for stop_word in ['стоп', 'end', 'ноль']):
            return 0
            
        numbers = re.findall(r'\d+[\.,]?\d*', val)
        if numbers:
            num_str = numbers[0].replace(',', '.')
            try:
                return float(num_str)
            except:
                return 0
        else:
            return 0
            
    return 0

In [54]:
financial_clean = financial_data.copy()
for col in month_columns:
    financial_clean[col] = financial_clean[col].apply(clean_numeric_value_alt)

financial_agg = financial_clean.groupby(['id', 'Account'])[month_columns].sum().reset_index()

print("Финансовые данные после агрегации:")
financial_agg.head()

Финансовые данные после агрегации:


,id,Account,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024
0,15,Иванова Мария Сергеевна,438.0,438.0,102.0,102.0,102.0,138.0,138.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16,Иванова Мария Сергеевна,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31,Васильев Артем Александрович,55.0,55.0,0.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,46.0
3,39,Попова Екатерина Николаевна,137.0,137.0,149.0,149.0,149.0,149.0,149.0,149.0,149.0,149.0,149.0,149.0,149.0,149.0,0.0,0.0
4,42,Васильев Артем Александрович,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
month_col_mapping = {}
for col in month_columns:
    year, month_num = month_to_number(col)
    month_col_mapping[(year, month_num)] = col

list(month_col_mapping.items())[:5]

[((2022, 11), 'Ноябрь 2022'),
 ((2022, 12), 'Декабрь 2022'),
 ((2023, 1), 'Январь 2023'),
 ((2023, 2), 'Февраль 2023'),
 ((2023, 3), 'Март 2023')]

In [56]:
def calculate_prolongation_ratios(prolongations_df, financial_df, month_mapping):
    results = []
    
    for month in range(1, 13):
        current_year = 2023
        current_month = month
        
        if current_month == 1:
            prev_month = 12
            prev_year = current_year - 1
            two_months_ago_month = 11
            two_months_ago_year = current_year - 1
        elif current_month == 2:
            prev_month = 1
            prev_year = current_year
            two_months_ago_month = 12
            two_months_ago_year = current_year - 1
        else:
            prev_month = current_month - 1
            prev_year = current_year
            two_months_ago_month = current_month - 2
            two_months_ago_year = current_year
        
        current_col = month_mapping.get((current_year, current_month))
        prev_col = month_mapping.get((prev_year, prev_month))
        two_months_ago_col = month_mapping.get((two_months_ago_year, two_months_ago_month))
        
        if not all([current_col, prev_col, two_months_ago_col]):
            continue
        
        projects_prev_month = prolongations_df[
            (prolongations_df['year'] == prev_year) & 
            (prolongations_df['month_num'] == prev_month)
        ]
        
        projects_two_months_ago = prolongations_df[
            (prolongations_df['year'] == two_months_ago_year) & 
            (prolongations_df['month_num'] == two_months_ago_month)
        ]
        
        managers = prolongations_df['AM'].unique()
        
        for manager in managers:
            manager_prev_projects = projects_prev_month[projects_prev_month['AM'] == manager]
            
            total_shipment_prev = 0
            prolonged_shipment_prev = 0
            
            for _, project in manager_prev_projects.iterrows():
                project_id = project['id']
                project_data = financial_df[financial_df['id'] == project_id]
                
                if len(project_data) > 0:
                    last_month_shipment = project_data[prev_col].iloc[0] if prev_col in project_data.columns else 0
                    current_month_shipment = project_data[current_col].iloc[0] if current_col in project_data.columns else 0
                    
                    total_shipment_prev += last_month_shipment
                    if current_month_shipment > 0:
                        prolonged_shipment_prev += current_month_shipment
            
            ratio1 = prolonged_shipment_prev / total_shipment_prev if total_shipment_prev > 0 else 0
            
            manager_two_months_projects = projects_two_months_ago[projects_two_months_ago['AM'] == manager]
            
            total_shipment_two = 0
            prolonged_shipment_two = 0
            
            for _, project in manager_two_months_projects.iterrows():
                project_id = project['id']
                project_data = financial_df[financial_df['id'] == project_id]
                
                if len(project_data) > 0:
                    last_month_shipment = project_data[two_months_ago_col].iloc[0] if two_months_ago_col in project_data.columns else 0
                    next_month_shipment = project_data[prev_col].iloc[0] if prev_col in project_data.columns else 0
                    current_month_shipment = project_data[current_col].iloc[0] if current_col in project_data.columns else 0
                    
                    if next_month_shipment == 0:
                        total_shipment_two += last_month_shipment
                        if current_month_shipment > 0:
                            prolonged_shipment_two += current_month_shipment
            
            ratio2 = prolonged_shipment_two / total_shipment_two if total_shipment_two > 0 else 0
            
            total_ratio = (ratio1 + ratio2) / 2 if (ratio1 + ratio2) > 0 else 0
            
            results.append({
                'Year': current_year,
                'Month': current_month,
                'Manager': manager,
                'First_Month_Ratio': ratio1,
                'Second_Month_Ratio': ratio2,
                'Total_Ratio': total_ratio,
                'Projects_Count_Prev': len(manager_prev_projects),
                'Projects_Count_Two_Months': len(manager_two_months_projects)
            })
    
    return pd.DataFrame(results)

In [57]:
ratios_df = calculate_prolongation_ratios(prolongations_clean, financial_agg, month_col_mapping)
print(f"\nРезультаты расчета:")
ratios_df.head(10)


Результаты расчета:


,Year,Month,Manager,First_Month_Ratio,Second_Month_Ratio,Total_Ratio,Projects_Count_Prev,Projects_Count_Two_Months
0,2023,1,Васильев Артем Александрович,0.556346,0.000000,0.278173,22,12
1,2023,1,Михайлов Андрей Сергеевич,0.684953,0.000000,0.342476,10,4
2,2023,1,Соколова Анастасия Викторовна,0.426049,0.675926,0.550987,9,5
3,2023,1,Иванова Мария Сергеевна,0.270424,0.000000,0.135212,16,1
4,2023,1,Попова Екатерина Николаевна,0.437500,0.000000,0.218750,6,2
5,2023,1,Смирнова Ольга Владимировна,0.743243,0.000000,0.371622,3,0
6,2023,1,Кузнецов Михаил Иванович,1.269231,0.000000,0.634615,1,0
7,2023,1,без А/М,0.000000,0.000000,0.000000,0,0
8,2023,1,Петрова Анна Дмитриевна,0.000000,0.000000,0.000000,0,0
9,2023,1,Федорова Марина Васильевна,0.000000,0.000000,0.000000,0,0


In [58]:
def calculate_summary_stats(ratios_df):
    manager_stats = ratios_df.groupby('Manager').agg({
        'First_Month_Ratio': ['mean', 'count'],
        'Second_Month_Ratio': 'mean',
        'Total_Ratio': 'mean',
        'Projects_Count_Prev': 'sum',
        'Projects_Count_Two_Months': 'sum'
    }).round(3)
    
    monthly_stats = ratios_df.groupby('Month').agg({
        'First_Month_Ratio': 'mean',
        'Second_Month_Ratio': 'mean', 
        'Total_Ratio': 'mean'
    }).round(3)
    
    return manager_stats, monthly_stats

In [59]:
manager_stats, monthly_stats = calculate_summary_stats(ratios_df)

print("\nСтатистика по менеджерам:")
manager_stats


Статистика по менеджерам:


First_Month_Ratio       Second_Month_Ratio  \
                                           mean count               mean   
Manager                                                                    
Васильев Артем Александрович              0.554    12              0.135   
Иванова Мария Сергеевна                   0.334    12              0.000   
Кузнецов Михаил Иванович                  0.347    12              0.000   
Михайлов Андрей Сергеевич                 0.660    12              0.000   
Петрова Анна Дмитриевна                   0.093    12              0.000   
Попова Екатерина Николаевна               0.383    12              0.125   
Смирнова Ольга Владимировна               0.457    12              0.305   
Соколова Анастасия Викторовна             0.579    12              0.074   
Федорова Марина Васильевна                0.000    12              0.000   
без А/М                                   0.000    12              0.000   

                              Total_Ratio Projects_Count_Prev  \
                                     mean                 sum   
Manager                                                         
Васильев Артем Александрович        0.344                 110   
Иванова Мария Сергеевна             0.167                  47   
Кузнецов Михаил Иванович            0.174                  15   
Михайлов Андрей Сергеевич           0.330                  26   
Петрова Анна Дмитриевна             0.046                   1   
Попова Екатерина Николаевна         0.254                  61   
Смирнова Ольга Владимировна         0.381                  51   
Соколова Анастасия Викторовна       0.327                  72   
Федорова Марина Васильевна          0.000                   0   
без А/М                             0.000                   1   

                              Projects_Count_Two_Months  
                                                    sum  
Manager                                                  
Васильев Артем Александрович                        115  
Иванова Мария Сергеевна                              48  
Кузнецов Михаил Иванович                              9  
Михайлов Андрей Сергеевич                            29  
Петрова Анна Дмитриевна                               0  
Попова Екатерина Николаевна                          59  
Смирнова Ольга Владимировна                          39  
Соколова Анастасия Викторовна                        68  
Федорова Марина Васильевна                            0  
без А/М                                               1

In [60]:
print("\nСтатистика по месяцам:")
print(monthly_stats)


Статистика по месяцам:
       First_Month_Ratio  Second_Month_Ratio  Total_Ratio
Month                                                    
1                  0.439               0.068        0.253
2                  0.314               0.040        0.177
3                  0.510               0.188        0.349
4                  0.406               0.030        0.218
5                  0.368               0.000        0.184
6                  0.155               0.006        0.080
7                  0.296               0.102        0.199
8                  0.241               0.158        0.199
9                  0.269               0.000        0.134
10                 0.532               0.070        0.301
11                 0.211               0.019        0.115
12                 0.346               0.087        0.217


In [61]:
with pd.ExcelWriter('prolongation_analysis_report.xlsx') as writer:
    ratios_df.to_excel(writer, sheet_name='Monthly_Ratios', index=False)
    
    manager_stats_flattened = manager_stats.copy()
    manager_stats_flattened.columns = ['_'.join(col).strip() for col in manager_stats_flattened.columns.values]
    manager_stats_flattened.reset_index(inplace=True)
    manager_stats_flattened.to_excel(writer, sheet_name='Manager_Stats', index=False)
    
    monthly_stats.reset_index(inplace=True)
    monthly_stats.to_excel(writer, sheet_name='Monthly_Stats', index=False)
    
    pivot_table = ratios_df.pivot_table(
        index='Month',
        columns='Manager', 
        values='Total_Ratio',
        aggfunc='mean'
    ).fillna(0).round(3)
    pivot_table.to_excel(writer, sheet_name='Pivot_Table')

print("Анализ завершен! Результаты сохранены в prolongation_analysis_report.xlsx")

Анализ завершен! Результаты сохранены в prolongation_analysis_report.xlsx
